# load package

In [8]:
import numpy as np
import os
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 函数 analyze_pca_themes 解释

## 原代码

In [ ]:
def analyze_pca_themes(embeddings_path=r"V:\20240920\theme_analysis_act3301\text_embeddings_clean_lb2.npy", 
                       data_path=r"V:\20240920\theme_analysis_act3301\act3301_processed_data_clean.csv",
                       n_components = 34,   
                       n_top_texts=5):
    # Load data
    print("Loading data...")
    embeddings = np.load(embeddings_path)
    df = pd.read_csv(data_path)

    # compare length of embedding and data file 
    print("\n --- compare length of embedding and data file ---")
    print(f"Embeddings shape: {embeddings.shape}")
    print(f"length of data file: {len(df['letter'])}")
    
    # Check data length consistency
    if embeddings.shape[0] != len(df):
        raise ValueError("The number of embeddings does not match the number of texts in the data file.")
  
    # Standardize embeddings
    print("\n--- Standardizing embeddings...")
    scaler = StandardScaler()
    scaled_embeddings = scaler.fit_transform(embeddings)
   
    # Apply PCA
    print(f"\n--- Applying PCA with {n_components} components...")
    pca = PCA(n_components=n_components)
    pca_result = pca.fit_transform(scaled_embeddings)
   
    print(f"\n --- Total explained variance: {sum(pca.explained_variance_ratio_):.4f}")
    
    # Print explained variance ratio per component
    print("\n--- Explained Variance Ratio per Component ---")
    for i, ratio in enumerate(pca.explained_variance_ratio_):
        print(f"Component {i+1}: {ratio:.4f}")
   
    # Analyze themes
    print("\n\n --- Analyzing themes in each component...")
    themes = {}
    for i in range(n_components):
        # Get component scores
        scores = pca_result[:, i]
        
        # Get top and bottom texts
        top_indices = np.argsort(scores)[-n_top_texts:]
        bottom_indices = np.argsort(scores)[:n_top_texts]
        
        # Add statistics
        themes[f'Component_{i+1}'] = {
            'positive': [(df['letter'].iloc[idx], scores[idx]) 
                         for idx in reversed(top_indices)],
            'negative': [(df['letter'].iloc[idx], scores[idx]) 
                         for idx in bottom_indices],
            'score_stats': {
                'mean': np.mean(scores),
                'std': np.std(scores),
                'min': np.min(scores),
                'max': np.max(scores)
            }
        }
   
    return pca_result, themes

## 拆解代码，详细分析

### 函数的输入参数  

- **embeddings_path**: 文本嵌入的文件路径，默认值是：<font color=darkred>r"V:\20240920\theme_analysis_act3301\text_embeddings_clean_lb2.npy"</font>
- **data_path**: 处理后的数据文件路径，默认值是：<font color=darkred>"V:\20240920\theme_analysis_act3301\act3301_processed_data_clean.csv"</font>
- **n_components**: PCA中要保留的主成分数量，默认值是：<font color=darkred> 34 </font>
- **n_top_texts**: 每个主成分中要分析的文本数量，默认是：<font color=darkred> 5 </font>

In [10]:
embeddings_path=r"V:\20240920\theme_analysis_act3301\text_embeddings_clean_lb2.npy"  
data_path=r"V:\20240920\theme_analysis_act3301\act3301_processed_data_clean.csv" 
n_components = 34   
n_top_texts=5

### 数据加载

- **embeddings**: 从 embeddings_path 加载的文本嵌入，是一个二维的NumPy数组，每一行代表一个文本的嵌入向量  
- **df**: 从 data_path 加载的CSV文件，通常是一个DataFrame，包含处理后的文本数据  

In [72]:
# Load data
print("...Loading numpy embedding data...")          # 为了理解代码添加
embeddings = np.load(embeddings_path)
print(f"The shape of embeddings is : {embeddings.shape} ")   # 为了理解代码添加
print(f"The row number of the embeddings is : {embeddings.shape[0]} ")   # 为了理解代码添加
print(f"The column number of the embeddings is : {embeddings.shape[1]} ")   # 为了理解代码添加

print("\n...Loading csv data...")          # 为了理解代码添加
df = pd.read_csv(data_path)
print(f"The shape of csv data is : {df.shape} ")   # 为了理解代码添加
print(f"The row number of the csv data is : {df.shape[0]} ")   # 为了理解代码添加
print(f"The column number of the csv data is : {df.shape[1]} ")   # 为了理解代码添加

...Loading numpy embedding data...
The shape of embeddings is : (5586, 768) 
The row number of the embeddings is : 5586 
The column number of the embeddings is : 768 

...Loading csv data...
The shape of csv data is : (5586, 9) 
The row number of the csv data is : 5586 
The column number of the csv data is : 9 


### 数据长度一致性检查

- 打印 Embeddings 的形状和 df 中 letter 列的长度，用于比较两者的行数是否一致。
- 如果 Embeddings 的行数与 df 中的行数不一致，就抛出 ValueError 异常，提示嵌入数量与文本数量不匹配。然后退出运行。

In [73]:
# compare length of embedding and data file 
print("\n --- compare length of embedding and data file ---")
print(f"Embeddings shape: {embeddings.shape}")
print(f"length of data file: {len(df['letter'])}")
    
# Check data length consistency
if embeddings.shape[0] != len(df):
    raise ValueError("The number of embeddings does not match the number of texts in the data file.")


 --- compare length of embedding and data file ---
Embeddings shape: (5586, 768)
length of data file: 5586


### 对Embeddings进行标准化处理

- 使用 StandardScaler 对嵌入矩阵进行标准化处理，使得每个特征的均值为0，方差为1。
- **注意**: <font color=darkred> StandardScaler() 是按列进行标准化的，而不是按行。 </font>

In [74]:
# Standardize embeddings
print("\n--- Standardizing embeddings...")
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(embeddings)


--- Standardizing embeddings...


### 进行主成分分析（PCA）

- 使用 PCA 对标准化后的 embeddings 进行降维，保留由输入参数指定的 n_components 个主成分。
- print打印所有主成分解释的总方差比例，保留4位小数。

In [75]:
# Apply PCA
print(f"\n--- Applying PCA with {n_components} components...")
pca = PCA(n_components=n_components)
pca_result = pca.fit_transform(scaled_embeddings)
   
print(f"\n --- Total explained variance: {sum(pca.explained_variance_ratio_):.4f}")

print(f"\n --- The pca_result shape is: {pca_result.shape}")          # 为了理解代码添加


--- Applying PCA with 34 components...

 --- Total explained variance: 0.7025

 --- The pca_result shape is: (5586, 34)


### 每个主成分解释的方差比例

- print每个主成分的解释的方差比例，保留4位小数。

In [76]:
# Print explained variance ratio per component
print("\n--- Explained Variance Ratio per Component ---")
for i, ratio in enumerate(pca.explained_variance_ratio_):
    print(f"Component {i+1}: {ratio:.4f}")


--- Explained Variance Ratio per Component ---
Component 1: 0.1112
Component 2: 0.0743
Component 3: 0.0731
Component 4: 0.0517
Component 5: 0.0388
Component 6: 0.0344
Component 7: 0.0289
Component 8: 0.0241
Component 9: 0.0211
Component 10: 0.0188
Component 11: 0.0175
Component 12: 0.0157
Component 13: 0.0147
Component 14: 0.0144
Component 15: 0.0128
Component 16: 0.0121
Component 17: 0.0117
Component 18: 0.0111
Component 19: 0.0098
Component 20: 0.0097
Component 21: 0.0093
Component 22: 0.0089
Component 23: 0.0086
Component 24: 0.0081
Component 25: 0.0079
Component 26: 0.0072
Component 27: 0.0071
Component 28: 0.0065
Component 29: 0.0063
Component 30: 0.0057
Component 31: 0.0057
Component 32: 0.0053
Component 33: 0.0052
Component 34: 0.0050


### Analyze themes 段详解

**对每个主成分进行主题分析：**
  
- **scores**: 当前主成分的得分。  
- **top_indices**: 得分最高的 n_top_texts 个文本的索引。  
- **bottom_indices**: 得分最低的 n_top_texts 个文本的索引。  
- **positive**: 得分最高的文本及其得分。  
- **negative**: 得分最低的文本及其得分。  
- **score_stats**: 当前主成分得分的统计信息（均值、标准差、最小值、最大值）。  

In [77]:
# Analyze themes
print("\n\n --- Analyzing themes in each component...")
themes = {}
for i in range(n_components):
    # Get component scores
    scores = pca_result[:, i]   # 提取的是第 i 个主成分的所有样本得分（即每个样本在第 i 个主成分上的投影值
        
    # Get top and bottom texts
    top_indices = np.argsort(scores)[-n_top_texts:]
    bottom_indices = np.argsort(scores)[:n_top_texts]
        
    # Add statistics
    themes[f'Component_{i+1}'] = {
        'positive': [(df['letter'].iloc[idx], scores[idx]) 
                        for idx in reversed(top_indices)],
        'negative': [(df['letter'].iloc[idx], scores[idx]) 
                        for idx in bottom_indices],
        'score_stats': {
                'mean': np.mean(scores),
                'std': np.std(scores),
                'min': np.min(scores),
                'max': np.max(scores)
            }
        }



 --- Analyzing themes in each component...


#### 代码中 `scores = pca_result[:, i]` 的含义 

**pca_result[:, i]** 表示从 `pca_result` 矩阵中提取第 `i` 列的所有行数据。具体来说：  
  
**语法解释**:  
  
- **pca_result** 是一个二维数组（或矩阵），形状为 **(n_samples, n_components)**，其中：  
- - **n_samples** 是样本的数量（即文本的数量）。  
- - **n_components** 是主成分的数量（即 PCA 降维后的  维
  度）。  

- **[:, i]** 是 NumPy 的片操作，表示：

- - **:** 表示选择所有行。
- - **i** 表示选择第 i 列。

**具体含义**
- **pca_result[:, i]** 提取的是第 **i** 个主成分的所有样本得分（即每个样本在第 **i** 个主成分上的投影值）。  
- 结果是一个一维数组，形状为 **(n_samples,)**，表示每个样本在第 **i** 个主成分上的得分。

In [78]:
print(f"The shape of 'pca_result[:, 1]' is: {pca_result[:, 1].shape}")

The shape of 'pca_result[:, 1]' is: (5586,)


#### 代码中 `top_indices = np.argsort(scores)[-n_top_texts:]` 的含义

**top_indices = np.argsort(scores)[-n_top_texts:]** 的作用是找到得分最高的 n_top_texts 个文本的**索引值**。以下是对这行代码的详细解释：排序后的索引。

 <font color=darkred size=3.5> 1. 函数 `np.argsort()` 功能及效果演示  </font>
- **功能**：对 scores 数组进行排序，并返回排序后的**索引值**。
- **排序方式**：默认是升序排序（从小到大）。
- **返回值**：一个与 scores 形状相同的数组，表示排序后的**索引值**。

In [79]:
# demo code for 理解代码
scores_demo = [3.2, 1.5, 4.7, 2.8, 0.9]
np.argsort(scores_demo) 

array([4, 1, 3, 0, 2], dtype=int64)

**上面的代码结果解释**：
- 序号为 4 的4 个元素（0.9）是最小的，排在第   
- 序号为 1 的第 1 个元素（1.5）是第二小的，排在  
- 序号为 3 的元素（2.8）是第三小的，排在第 3 位  
- 序号为 0 的元素
第 0 个元素（3.2）是第四小  
- 序号为 2 的元素

第 2 个元素（4.7）是  大的，排在第 5 位。的，排在第 5 位。

<font color=darkred size=3.5> 2. `[-n_top_texts:]`功能 </font>  

**功能**:  从排序后的索引中提取最后 `n_top_texts` 个索引 

**解释**：

- `n_top_texts` 表示从数组的末尾开始计数，提取最后 n_top_texts个元素。

- `: `表示提取这些元素的所有内容。

In [80]:
scores = pca_result[:, 0]
top_indices = np.argsort(scores)[-n_top_texts:]
top_indices

array([5514, 1993, 5440, 5475, 3726], dtype=int64)

<font color=darkred size=3.5> 3. `top_indices = np.argsort(scores)[-n_top_texts:]` 的作用总结 </font> 

`top_indices = np.argsort(scores)[-n_top_texts:]` 的作用是: 

1. 对 scores 进行升序排序，找到得分最高的文本索引值。  
2. 然后从排序后的索引值列表中提取最后 n_top_texts 个索引值，这些索引值可以用来作为行数，查找 df 中对应行的`letter`列的内容，它们就是得分最高的文本。

<font color=darkred> 本例中，pca_result的shape是(5586, 34)，它的每一行对应 df （也就是csv文件）中的每一行。</font>

<font color=darkred size=3.5> 4. 总结 </font>   
`top_indices = np.argsort(scores)[-n_top_texts:]` 的作用是找到得分最高的 n_top_texts 个文本的索引。  
它是通过先对 scores 进行升序排序，然后提取排序后索引的最后 n_top_texts 个元素实现的。

#### 从csv文件中取出对应的letter内容的方法

代码中使用从 pca_result 中得到的索引值列表 top_indices 和 bottom_indices ，遍历它们，再去 df （也就是csv文件）中逐一寻找对应的行，然后找到 letter 的内容。

**逐一寻找对应的行，然后找到该行 letter 内容的演示代码如下**

In [81]:
df['letter'].iloc[2]

'.senator it is in your power to re introduce act3301 back into discussion. women should have the right to equal liberties to be in position to take on roles as any man should be able to. it is unfair of you to look at the issue no matter what your stance is.'

#### 为每个主成分创建一个字典，记录该主成分的分析结果。

In [82]:
    # Analyze themes
    print("\n\n --- Analyzing themes in each component...")
    themes = {}
    for i in range(n_components):
        # Get component scores
        scores = pca_result[:, i]
        
        # Get top and bottom texts
        top_indices = np.argsort(scores)[-n_top_texts:]
        bottom_indices = np.argsort(scores)[:n_top_texts]
        
        # Add statistics
        themes[f'Component_{i+1}'] = {
            'positive': [(df['letter'].iloc[idx], scores[idx]) 
                         for idx in reversed(top_indices)],
            'negative': [(df['letter'].iloc[idx], scores[idx]) 
                         for idx in bottom_indices],
            'score_stats': {
                'mean': np.mean(scores),
                'std': np.std(scores),
                'min': np.min(scores),
                'max': np.max(scores)
            }
        }



 --- Analyzing themes in each component...


具体来说，它包含以下三个部分：

1. **positive**  
- **作用**：记录得分最高的 n_top_texts 个文本及其对应的得分。

- **实现**：

- - `top_indices` 是得分最高的文本索引。
  - `reversed(top_indices)` 将 `top_indices` 反转，使得得分最高的文本排在最前面。
  - `[(df['letter'].iloc[idx], scores[idx]) for idx in reversed(top_indices)]` 是一个列表推导式，遍历反转后的 `top_indices`，提取每个索引对应的文本内容`（df['letter'].iloc[idx]）`和得分`（scores[idx]）`。

**示例代码如下**:

In [83]:
# demo code
demo_top_indices = [0, 3]  # 得分最高的文本索引
demo_df = {'letter': ['text1', 'text2', 'text3', 'text4']}  # 文本内容
demo_scores = [1.5, 0.8, 1.2, 2.5]  # 得分

positive_texts = [(demo_df['letter'][idx], demo_scores[idx]) for idx in reversed(demo_top_indices)]
print(positive_texts)

[('text4', 2.5), ('text1', 1.5)]


2. **negative**
- **作用**：记录得分最低的 n_top_texts 个文本及其对应的得分。

- **实现**：
- - `bottom_indices` 是得分最低的文本索引。
- - `[(df['letter'].iloc[idx], scores[idx]) for idx in bottom_indices]` 是一个列表推导式，遍历 `bottom_indices`，提取每个索引对应的文本内容和得分。

3. **score_stats**
- **作用**：记录当前主成分得分的统计信息。

- **实现**：
- `np.mean(scores)`：计算得分的均值。
- `np.std(scores)`：计算得分的标准差。
- `np.min(scores)`：计算得分的最小值。
- `np.max(scores)`：计算得分的最大值。

#### 构造一个 themes 的演示字典

##### 构建 themes 字典的各个元素

In [84]:
# demo code
demo_df = {'letter': ['text1', 'text2', 'text3', 'text4', 'text5']}  # 文本内容
demo_scores = [0.5, 0.8, -0.7, 0.9, -0.3]  # 得分
demo_top_indices = [1, 3]  # 得分最高的文本索引
demo_bottom_indices = [2, 4]  # 得分最低的文本索引

# 构造 positive_texts
positive_texts = [(demo_df['letter'][idx], demo_scores[idx]) for idx in reversed(demo_top_indices)]
print(positive_texts)

# 构造 negative_texts。
# 注意：原始代码中此处没有做 reversed 操作，因为程序的运行环境下，此处得到的 score 都是负值
negative_texts = [(demo_df['letter'][idx], demo_scores[idx]) for idx in demo_bottom_indices]
print(negative_texts)

# 构造 score_stats
demo_score_stats = {
    'mean': np.mean(demo_scores),
    'std': np.std(demo_scores),
    'min': np.min(demo_scores),
    'max': np.max(demo_scores)
}
print(demo_score_stats)

[('text4', 0.9), ('text2', 0.8)]
[('text3', -0.7), ('text5', -0.3)]
{'mean': 0.24, 'std': 0.63118935352238, 'min': -0.7, 'max': 0.9}


##### 手工构造一个 themes 的演示字典

In [85]:
# demo code
demo_df = {'letter': ['text1', 'text2', 'text3', 'text4', 'text5']}  # 文本内容
demo_scores = [0.5, 0.8, -0.7, 0.9, -0.3]  # 得分
demo_top_indices = [1, 3]  # 得分最高的文本索引
demo_bottom_indices = [2, 4]  # 得分最低的文本索引

themes_demo = {}
themes_demo[0] = {
            'positive': [(demo_df['letter'][idx], demo_scores[idx]) for idx in reversed(demo_top_indices)],
            'negative': [(demo_df['letter'][idx], demo_scores[idx]) for idx in demo_bottom_indices],
            'score_stats': {
                'mean': np.mean(demo_scores),
                'std': np.std(demo_scores),
                'min': np.min(demo_scores),
                'max': np.max(demo_scores)
            }
        }

themes_demo[1] = {
            'positive': [(demo_df['letter'][idx], demo_scores[idx]) for idx in reversed(demo_top_indices)],
            'negative': [(demo_df['letter'][idx], demo_scores[idx]) for idx in demo_bottom_indices],
            'score_stats': {
                'mean': np.mean(demo_scores),
                'std': np.std(demo_scores),
                'min': np.min(demo_scores),
                'max': np.max(demo_scores)
            }
        }

themes_demo

{0: {'positive': [('text4', 0.9), ('text2', 0.8)],
  'negative': [('text3', -0.7), ('text5', -0.3)],
  'score_stats': {'mean': 0.24,
   'std': 0.63118935352238,
   'min': -0.7,
   'max': 0.9}},
 1: {'positive': [('text4', 0.9), ('text2', 0.8)],
  'negative': [('text3', -0.7), ('text5', -0.3)],
  'score_stats': {'mean': 0.24,
   'std': 0.63118935352238,
   'min': -0.7,
   'max': 0.9}}}

## 写入xlsx文件代码段详解

**先构造一个 themes 的演示字典**

In [86]:
# 先构造一个 themes 的演示字典
demo_df = {'letter': ['text1', 'text2', 'text3', 'text4', 'text5']}  # 文本内容
demo_scores = [0.5, 0.8, -0.7, 0.9, -0.3]  # 得分
demo_top_indices = [1, 3]  # 得分最高的文本索引
demo_bottom_indices = [2, 4]  # 得分最低的文本索引

themes_demo = {}
themes_demo[0] = {
            'positive': [(demo_df['letter'][idx], demo_scores[idx]) for idx in reversed(demo_top_indices)],
            'negative': [(demo_df['letter'][idx], demo_scores[idx]) for idx in demo_bottom_indices],
            'score_stats': {
                'mean': np.mean(demo_scores),
                'std': np.std(demo_scores),
                'min': np.min(demo_scores),
                'max': np.max(demo_scores)
            }
        }

themes_demo[1] = {
            'positive': [(demo_df['letter'][idx], demo_scores[idx]) for idx in reversed(demo_top_indices)],
            'negative': [(demo_df['letter'][idx], demo_scores[idx]) for idx in demo_bottom_indices],
            'score_stats': {
                'mean': np.mean(demo_scores),
                'std': np.std(demo_scores),
                'min': np.min(demo_scores),
                'max': np.max(demo_scores)
            }
        }

themes_demo.items()

dict_items([(0, {'positive': [('text4', 0.9), ('text2', 0.8)], 'negative': [('text3', -0.7), ('text5', -0.3)], 'score_stats': {'mean': 0.24, 'std': 0.63118935352238, 'min': -0.7, 'max': 0.9}}), (1, {'positive': [('text4', 0.9), ('text2', 0.8)], 'negative': [('text3', -0.7), ('text5', -0.3)], 'score_stats': {'mean': 0.24, 'std': 0.63118935352238, 'min': -0.7, 'max': 0.9}})])

### 1. `for component, theme_data in themes.items()` 的作用 

`themes.items()`代表遍历并提取字典themes中的所有元素

**演示代码如下**:

In [87]:
for component, theme_data in themes_demo.items():
    print(component)
    print(theme_data)

0
{'positive': [('text4', 0.9), ('text2', 0.8)], 'negative': [('text3', -0.7), ('text5', -0.3)], 'score_stats': {'mean': 0.24, 'std': 0.63118935352238, 'min': -0.7, 'max': 0.9}}
1
{'positive': [('text4', 0.9), ('text2', 0.8)], 'negative': [('text3', -0.7), ('text5', -0.3)], 'score_stats': {'mean': 0.24, 'std': 0.63118935352238, 'min': -0.7, 'max': 0.9}}


### 2. 构建3个 df，并写入excel文件的不同sheet 

**演示代码如下**:

In [88]:
output_path = r"V:\20240920\theme_analysis_act3301\xlsxwriter_demo.xlsx"
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    for component, theme_data in themes_demo.items():
        # Create a DataFrame for positive examples
        positive_df = pd.DataFrame(theme_data['positive'], columns=['Text', 'Score'])
        positive_df.to_excel(writer, sheet_name=f"{component}_Positive", index=False)
            
        # Create a DataFrame for negative examples
        negative_df = pd.DataFrame(theme_data['negative'], columns=['Text', 'Score'])
        negative_df.to_excel(writer, sheet_name=f"{component}_Negative", index=False)
            
        # Add statistics to the Excel file
        stats_df = pd.DataFrame([theme_data['score_stats']])
        stats_df.to_excel(writer, sheet_name=f"{component}_Stats", index=False)

<font color=darkred size=3.5> 主题分析 </font>